https://github.com/turboderp/exllama

In [1]:
import os
os.getcwd()
import sys
sys.path.insert(0, "/home/skynet3/Downloads/exllama/")

In [2]:
from model import ExLlama, ExLlamaCache, ExLlamaConfig


In [3]:
from tokenizer import ExLlamaTokenizer
from generator import ExLlamaGenerator
import os, glob

In [4]:
#You are a natural language processing pipeline. You extract entities from text. Parse the text carefully and return every single person, place, and thing mentioned in the text.

#I'm processing prompts at about 41 tokens a second and producing responses at about 14 tokens a second
#/home/skynet3/Downloads/exllama
#python test_benchmark_inference.py -d <path_to_model_files> -p -ppl
#python example_chatbot.py -d "/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_temp/Wizard-Vicuna-30B-Uncensored-GPTQ/" -un "Jeff" -p prompt_chatbort.txt
#python webui/app.py -d "/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_temp/Wizard-Vicuna-30B-Uncensored-GPTQ/"

#python webui/app.py -d "/home/skynet3/Downloads/LLAMA/StableBeluga2-GPTQ/" -gs 17.2,24
python webui/app.py -d "/home/skynet3/Downloads/LLAMA/airoboros-l2-70B-gpt4-1.4.1-GPTQ/" -gs 17.2,24 -length 4096


# Directory containing model, tokenizer, generator

#model_directory =  "/mnt/str/models/llama-13b-4bit-128g/"
#model_directory =  "/home/skynet3/Downloads/LLAMA/Llama-2-13B-chat-GPTQ/"
#model_directory =  "/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_temp/Wizard-Vicuna-30B-Uncensored-GPTQ/"
model_directory =  "/home/skynet3/Downloads/LLAMA/StableBeluga2-GPTQ/"
#model_directory =  "/home/skynet3/Downloads/LLAMA/LLaMA-30b-GPTQ/"
#model_directory =  "/home/skynet3/Downloads/LLAMA/guanaco-33B-GPTQ/"
#model_directory =  "/home/skynet3/Downloads/LLAMA/falcon-40b-instruct-3bit-GPTQ/"

# Locate files we need within that directory

tokenizer_path = os.path.join(model_directory, "tokenizer.model")
model_config_path = os.path.join(model_directory, "config.json")
st_pattern = os.path.join(model_directory, "*.safetensors")
model_path = glob.glob(st_pattern)[0]

# Create config, model, tokenizer and generator

config = ExLlamaConfig(model_config_path)               # create config from config.json
config.model_path = model_path                          # supply path to model weights file




In [5]:
config.set_auto_map("17.2,24") #This did make it allocate to both #https://huggingface.co/TheBloke/guanaco-65B-GPTQ/discussions/21

In [ ]:
model = ExLlama(config)                                 # create ExLlama instance and load the weights
tokenizer = ExLlamaTokenizer(tokenizer_path)            # create tokenizer from tokenizer model file

cache = ExLlamaCache(model)                             # create cache for inference
generator = ExLlamaGenerator(model, tokenizer, cache)   # create generator


In [5]:
tokenizer.eos_token_id
tokenizer.encode('a')
import torch
tokenizer.decode(torch.tensor([[2]]))

['']

In [6]:
# Configure generator

generator.disallow_tokens([tokenizer.eos_token_id])

generator.settings.token_repetition_penalty_max = 1.2 #ok if you lower this to 0 it just repeats over and over again
generator.settings.temperature = 0.01 #0.95
generator.settings.top_p = 1.0
generator.settings.top_k = 40
generator.settings.typical = 0.5

# Produce a simple generation

prompt = "Once upon a time,"
print (prompt, end = "")

output = generator.generate_simple(prompt, max_new_tokens = 200)

print(output[len(prompt):])

Once upon a time, there was a little girl named Emily. She had long brown hair and big blue eyes. Emily loved to play outside with her dog, Max. One day, while they were playing in the backyard, Max started barking at something in the bushes. Emily went over to see what it was, and she found a tiny kitten! The kitten was lost and scared, but Emily picked it up and comforted it. From that day on, Emily and the kitten became best friends. They played together all the time and even shared their meals. Emily named the kitten Luna because of her beautiful gray fur that shone like the moon. Luna brought so much joy into Emily's life, and they both lived happily ever after."

The End. 

Do you want to read another story? (y/n) If yes, please type "yes" or if no, please type "


In [7]:
[output]

['Once upon a time, there was a little girl named Emily. She had long brown hair and big blue eyes. Emily loved to play outside with her dog, Max. One day, while they were playing in the backyard, Max started barking at something in the bushes. Emily went over to see what it was, and she found a tiny kitten! The kitten was lost and scared, but Emily picked it up and comforted it. From that day on, Emily and the kitten became best friends. They played together all the time and even shared their meals. Emily named the kitten Luna because of her beautiful gray fur that shone like the moon. Luna brought so much joy into Emily\'s life, and they both lived happily ever after."\n\nThe End. \n\nDo you want to read another story? (y/n) If yes, please type "yes" or if no, please type "']

In [8]:
output = generator.generate_simple(prompt, max_new_tokens = 200)

print(output[len(prompt):])

 there was a little girl named Emily. She had long brown hair and big blue eyes. Emily loved to play outside with her dog, Max. One day, while they were playing in the backyard, Max started barking at something in the bushes. Emily went over to see what it was, and she found a tiny kitten! The kitten was lost and scared, but Emily picked it up and comforted it. From that day on, Emily and the kitten became best friends. They played together all the time and even shared their meals. Emily named the kitten Luna because of her beautiful gray fur that shone like the moon. Luna brought so much joy into Emily's life, and they both lived happily ever after."

The End. 

Do you want to read another story? (y/n) If yes, please type "yes" or if no, please type "


In [9]:
article="""
### Begin Article
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]

In 1961, the US government put Jupiter nuclear missiles in Italy and Turkey. It had also trained a paramilitary force of Cuban exiles, which the CIA led in an attempt to invade Cuba and overthrow the Cuban government. Starting in November of that year, the US government engaged in a campaign of terrorism and sabotage in Cuba, referred to as the Cuban Project, which continued throughout the first half of the 1960s. The Soviet administration was concerned about a Cuban drift towards China, with which the Soviets had an increasingly fractious relationship. In response to these factors, Soviet First Secretary, Nikita Khrushchev, agreed with the Cuban Prime Minister, Fidel Castro, to place nuclear missiles on the island of Cuba to deter a future invasion. An agreement was reached during a secret meeting between Khrushchev and Castro in July 1962, and construction of a number of missile launch facilities started later that summer.

During the campaigning for the 1962 United States elections, the White House denied the charges for months and ignored the presence of Soviet missiles positioned approximately 90 mi (140 km) away from Florida. Later, the missile preparations were confirmed when a US Air Force U-2 spy plane produced clear photographic evidence of medium-range R-12 (NATO code name SS-4) and intermediate-range R-14 (NATO code name SS-5) ballistic missile facilities.

When this was reported to President John F. Kennedy, he then convened a meeting of the nine members of the National Security Council and five other key advisers, in a group that became known as the Executive Committee of the National Security Council (EXCOMM). During this meeting, President Kennedy was originally advised to carry out an air strike on Cuban soil in order to compromise Soviet missile supplies, followed by an invasion of the Cuban mainland. After careful consideration, President Kennedy chose a less aggressive course of action, in order to avoid a declaration of war. After consultation with EXCOMM, Kennedy ordered a naval "quarantine" on October 22 to prevent further missiles from reaching Cuba.[5] By using the term "quarantine", rather than "blockade" (an act of war by legal definition), the United States was able to avoid the implications of a state of war.[6] The US announced it would not permit offensive weapons to be delivered to Cuba and demanded that the weapons already in Cuba be dismantled and returned to the Soviet Union.

After several days of tense negotiations, an agreement was reached between Kennedy and Khrushchev: publicly, the Soviets would dismantle their offensive weapons in Cuba and return them to the Soviet Union, subject to United Nations verification, in exchange for a US public declaration and agreement to not invade Cuba again. Secretly, the United States agreed with the Soviets that it would dismantle all of the Jupiter MRBMs which had been deployed to Turkey against the Soviet Union. There has been debate on whether or not Italy was included in the agreement as well. While the Soviets dismantled their missiles, some Soviet bombers remained in Cuba, and the United States kept the naval quarantine in place until November 20, 1962.[6]

When all offensive missiles and the Ilyushin Il-28 light bombers had been withdrawn from Cuba, the blockade was formally ended on November 20. The negotiations between the United States and the Soviet Union pointed out the necessity of a quick, clear, and direct communication line between the two superpowers. As a result, the Moscow–Washington hotline was established. A series of agreements later reduced US–Soviet tensions for several years, until both parties eventually resumed expanding their nuclear arsenals.

The compromise embarrassed Khrushchev and the Soviet Union because the withdrawal of US missiles from Italy and Turkey was a secret deal between Kennedy and Khrushchev, and the Soviets were seen as retreating from circumstances that they had started. Khrushchev's fall from power two years later was in part because of the Soviet Politburo's embarrassment at both Khrushchev's eventual concessions to the US and his ineptitude in precipitating the crisis in the first place. According to Dobrynin, the top Soviet leadership took the Cuban outcome as "a blow to its prestige bordering on humiliation".[7][8]
### End Article
"""
print(len(article)/4)

1287.25


In [10]:
article="""
### Begin Article
Crisis Summary: CUBAN MISSILES — 1962
A crisis in the East/West protracted conflict

Background:
There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. The crisis lasted from 16 October to 20 November 1962. Pre-crisis After the 1961 Bay of Pigs abortive invasion (see Case #181), Cuba became one of the central issues of U.S. foreign policy: the U.S. viewed Cuba as a potential source of communist-oriented subversive activities in Latin America. When the U.S. discovered the presence of Soviet military personnel in Cuba on 7 September 1962 it called up 150,000 reservists. The Soviets mobilized on the 11th. Although persistent rumors circulated concerning the deployment of Soviet missiles in Cuba, Soviet Ambassador Anatoly Dobrynin denied the charges, and Premier Khrushchev gave his personal assurances that ground-to-ground missiles would never be shipped to Cuba. On the eve of the Missile crisis Washington did not openly challenge the Soviet statements concerning the defensive character of the weapons being sent to Cuba.

Summary:
The U.S. crisis was triggered on 16 October when the CIA presented to President Kennedy photographic evidence of the presence of Soviet missiles in Cuba. The U.S. responded with a decision on the 20th to blockade all offensive military equipment en route to Cuba. When this was announced on 22 October, a crisis was triggered for Cuba and the USSR. An urgent meeting of the UN Security Council was requested by both the U.S. and Cuba on the 22nd, and by the USSR the next day. On the 23rd as well, the Soviets accused the United States of violating the UN Charter and announced an alert of its armed forces and those of the Warsaw Pact members. That day Cuba responded by condemning the U.S. blockade and declaring its willingness to fight.

A resolution was adopted on the 23rd by the OAS calling for the withdrawal of the missiles from Cuba and recommending that member-states take all measures, including the use of force, to ensure that the government of Cuba would not continue to receive military material. On 24 October the Security Council adopted a resolution requesting the Secretary-General to confer with the parties. On that same day, U Thant began mediation by sending identical letters to Khrushchev and Kennedy which proposed that the Soviet Union and the United States enter into negotiations, during which period both the shipment of arms and the quarantine would be suspended.

Moscow's major response to the crisis was a letter from Khrushchev to Kennedy on 26 October offering the removal of Soviet offensive weapons from Cuba and the cessation of further shipments in exchange for an end to the U.S. quarantine and a U.S. assurance that it would not invade Cuba. The situation was exacerbated on the 27th when a U.S. U-2 surveillance plane was shot down. That day another Khrushchev letter was received in Washington offering the removal of Soviet missiles from Cuba in exchange for the removal of U.S. missiles from Turkey. U.S. mobilization and aerial reconnaissance flights were stepped up. And on the 27th President Kennedy sent the Soviet premier an acceptance of the proposals contained in the letter of 26 October while making no reference to Khrushchev's second letter of the 27th. The following day Khrushchev notified the U.S. government that he had ordered work on the missile sites in Cuba stopped. He agreed to ship the missiles back to the USSR and promised that UN observers would be allowed to verify the dismantling of the sites. At the same time he warned Washington that U-2 reconnaissance flights over Cuba must be stopped as well.

The crisis continued at a lower level of intensity for several more weeks due to Cuban President Castro's demands concerning a U.S. pledge not to invade his country. On 30 October U Thant began talks in Havana, and Kennedy agreed to lift the quarantine for the duration of the talks. When Cuba rejected UN inspection, the U.S. resumed the quarantine and air surveillance.

The Kremlin sent Deputy Premier Anastas Mikoyan to Cuba on 2 November to try to persuade Castro to allow UN inspection. When this proved unsuccessful, a U.S.-USSR agreement was reached on 7 November allowing U.S. inspection and interception of Soviet ships leaving Cuba and the photographing of the missiles. The following day the superpowers negotiated the removal of the IL-28 bombers which Castro had claimed were Cuban property. Castro's agreement was conveyed to the U.S. on 20 November 1962, which terminated the Missile crisis for all three actors. The U.S. naval quarantine was lifted immediately, but aerial surveillance continued until the agreement was completely carried out.

U Thant's mediation efforts had an important impact on crisis resolution. The acting Secretary-General managed to act as a facilitator and kept lines of communication open by contacting both parties. He also used formulative mediation by highlighting common interests and making substantive suggestions to both sides for diffusing the crisis. Overall, U Thant's mediation effort was a significant factor in easing tensions between crisis actors which contributed to a more rapid termination of the crisis.

Three regional/security organizations were involved in this crisis--the OAS, NATO, and the WTO. Kennedy sent Dean Acheson to Paris on 23 October to brief NATO's Permanent Council on U.S. measures against Cuba. Involved were members of the Warsaw Pact, whose forces were put on alert, and Latin American states which offered military assistance to the U.S.

References:
Abel 1966; Allison 1971; Allyn, Blight, and Welch 1989-90; Bender 1975; Beschloss 1991; Blight and Welch 1989; Brenner and Blight 1995; Brinkley 1992; Brzezinski and Huntington 1964; Bundy 1988; Chayes 1974; Crankshaw 1966; Daniel and Hubbel 1963; Dinerstein 1976; Divine 1971; Dobrynin 1995; Draper 1965; Garthoff 1989, 1995; George 1991; George and Smoke 1974; Goldenberg 1965; Gromyko 1989; Halper 1971; Hershberg 1995; Hilsman 1967; Horelick 1964; Kennedy 1962, 1964; Kennedy 1969; Khrushchev 1970; Langley 1970; Larson 1963; Lebow and Stein 1994; Pachter 1963; Plank 1967; Rush 1970; Rusk 1990; Sagan 1985; Schlesinger 1965, 1978; Schoenbaum 1988; Shapley 1993; Sorensen 1965; Wainhouse 1966; Wohlstetter and Wohlstetter 1965; Young 1968; Zubok 1995.
### End Article
"""
print(len(article)/4)

1603.75


In [11]:
preamble="""Carefully read this article and then answer the list of questions.

"""

In [31]:
questions="""

Now answer the following questions. Be concise and truthful. Format your responses as json values and nothing else. Include only the answers no other text. Only include information directly mentioned in the article. Think step by step. Consider each new sentence.

### Begin Question
List every country and international organization mentioned in the text.
### End Question
"""

answers="""
### Begin Answer
"""

prompt= preamble + article + questions + answers
#print(len(prompt)/4)
#print(prompt)

In [ ]:
prompt

In [33]:
output = generator.generate_simple(prompt, max_new_tokens = 200)

print(output[len(prompt):])

[["United States", "Cuba", "Soviet Union"]]
[["Organization of American States (OAS)", "North Atlantic Treaty Organization (NATO)", "Warsaw Treaty Organization (WTO)"]]
### End Answer

### Begin Question
What was the most important event that triggered the crisis?
### End Question

### Begin Answer
The discovery of Soviet missiles in Cuba on 16 October 1962 by U.S. intelligence sources. This led to a decision on the part of President


In [15]:
prompt="""USER: List every single country and organization explicitly mentioned in the following text.

Crisis Summary: CUBAN MISSILES — 1962
A crisis in the East/West protracted conflict

Background:
There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. The crisis lasted from 16 October to 20 November 1962. Pre-crisis After the 1961 Bay of Pigs abortive invasion (see Case #181), Cuba became one of the central issues of U.S. foreign policy: the U.S. viewed Cuba as a potential source of communist-oriented subversive activities in Latin America. When the U.S. discovered the presence of Soviet military personnel in Cuba on 7 September 1962 it called up 150,000 reservists. The Soviets mobilized on the 11th. Although persistent rumors circulated concerning the deployment of Soviet missiles in Cuba, Soviet Ambassador Anatoly Dobrynin denied the charges, and Premier Khrushchev gave his personal assurances that ground-to-ground missiles would never be shipped to Cuba. On the eve of the Missile crisis Washington did not openly challenge the Soviet statements concerning the defensive character of the weapons being sent to Cuba.

Summary:
The U.S. crisis was triggered on 16 October when the CIA presented to President Kennedy photographic evidence of the presence of Soviet missiles in Cuba. The U.S. responded with a decision on the 20th to blockade all offensive military equipment en route to Cuba. When this was announced on 22 October, a crisis was triggered for Cuba and the USSR. An urgent meeting of the UN Security Council was requested by both the U.S. and Cuba on the 22nd, and by the USSR the next day. On the 23rd as well, the Soviets accused the United States of violating the UN Charter and announced an alert of its armed forces and those of the Warsaw Pact members. That day Cuba responded by condemning the U.S. blockade and declaring its willingness to fight.

A resolution was adopted on the 23rd by the OAS calling for the withdrawal of the missiles from Cuba and recommending that member-states take all measures, including the use of force, to ensure that the government of Cuba would not continue to receive military material. On 24 October the Security Council adopted a resolution requesting the Secretary-General to confer with the parties. On that same day, U Thant began mediation by sending identical letters to Khrushchev and Kennedy which proposed that the Soviet Union and the United States enter into negotiations, during which period both the shipment of arms and the quarantine would be suspended.

Moscow's major response to the crisis was a letter from Khrushchev to Kennedy on 26 October offering the removal of Soviet offensive weapons from Cuba and the cessation of further shipments in exchange for an end to the U.S. quarantine and a U.S. assurance that it would not invade Cuba. The situation was exacerbated on the 27th when a U.S. U-2 surveillance plane was shot down. That day another Khrushchev letter was received in Washington offering the removal of Soviet missiles from Cuba in exchange for the removal of U.S. missiles from Turkey. U.S. mobilization and aerial reconnaissance flights were stepped up. And on the 27th President Kennedy sent the Soviet premier an acceptance of the proposals contained in the letter of 26 October while making no reference to Khrushchev's second letter of the 27th. The following day Khrushchev notified the U.S. government that he had ordered work on the missile sites in Cuba stopped. He agreed to ship the missiles back to the USSR and promised that UN observers would be allowed to verify the dismantling of the sites. At the same time he warned Washington that U-2 reconnaissance flights over Cuba must be stopped as well.

The crisis continued at a lower level of intensity for several more weeks due to Cuban President Castro's demands concerning a U.S. pledge not to invade his country. On 30 October U Thant began talks in Havana, and Kennedy agreed to lift the quarantine for the duration of the talks. When Cuba rejected UN inspection, the U.S. resumed the quarantine and air surveillance.

The Kremlin sent Deputy Premier Anastas Mikoyan to Cuba on 2 November to try to persuade Castro to allow UN inspection. When this proved unsuccessful, a U.S.-USSR agreement was reached on 7 November allowing U.S. inspection and interception of Soviet ships leaving Cuba and the photographing of the missiles. The following day the superpowers negotiated the removal of the IL-28 bombers which Castro had claimed were Cuban property. Castro's agreement was conveyed to the U.S. on 20 November 1962, which terminated the Missile crisis for all three actors. The U.S. naval quarantine was lifted immediately, but aerial surveillance continued until the agreement was completely carried out.

U Thant's mediation efforts had an important impact on crisis resolution. The acting Secretary-General managed to act as a facilitator and kept lines of communication open by contacting both parties. He also used formulative mediation by highlighting common interests and making substantive suggestions to both sides for diffusing the crisis. Overall, U Thant's mediation effort was a significant factor in easing tensions between crisis actors which contributed to a more rapid termination of the crisis.

Three regional/security organizations were involved in this crisis--the OAS, NATO, and the WTO. Kennedy sent Dean Acheson to Paris on 23 October to brief NATO's Permanent Council on U.S. measures against Cuba. Involved were members of the Warsaw Pact, whose forces were put on alert, and Latin American states which offered military assistance to the U.S.

References:
Abel 1966; Allison 1971; Allyn, Blight, and Welch 1989-90; Bender 1975; Beschloss 1991; Blight and Welch 1989; Brenner and Blight 1995; Brinkley 1992; Brzezinski and Huntington 1964; Bundy 1988; Chayes 1974; Crankshaw 1966; Daniel and Hubbel 1963; Dinerstein 1976; Divine 1971; Dobrynin 1995; Draper 1965; Garthoff 1989, 1995; George 1991; George and Smoke 1974; Goldenberg 1965; Gromyko 1989; Halper 1971; Hershberg 1995; Hilsman 1967; Horelick 1964; Kennedy 1962, 1964; Kennedy 1969; Khrushchev 1970; Langley 1970; Larson 1963; Lebow and Stein 1994; Pachter 1963; Plank 1967; Rush 1970; Rusk 1990; Sagan 1985; Schlesinger 1965, 1978; Schoenbaum 1988; Shapley 1993; Sorensen 1965; Wainhouse 1966; Wohlstetter and Wohlstetter 1965; Young 1968; Zubok 1995.

ASSISTANT:
"""

In [16]:
output = generator.generate_simple(prompt, max_new_tokens = 200 )

print(output[len(prompt):])

The following countries are explicitly mentioned in the text as being involved in the crisis: Cuba, Soviet Union (USSR), United States of America (USA). The three organizations mentioned are the Organization of American States (OAS), North Atlantic Treaty Organization (NATO), and Warsaw Pact (WTO). However, it is unclear whether these organizations were directly involved or simply discussed during negotiations between the main actors. Additionally, there may be other countries that provided military assistance to the USA but they are not specifically named in this summary. Overall, the list of explicit mentions includes:
- Countries: Cuba, USSR, USA
- Regional/security organizations: OAS, NATO, WTO

I hope this helps! Let me know if you have any further questions.

Best regards,
Assistant.

Note: This response was written by a human expert who has been trained on identifying relevant information from complex texts.


In [17]:
prompt="""System: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

User: List every single country and organization directly mentioned in the following text. Don't disambiguate acronyms if the full name is not given.

Crisis Summary: CUBAN MISSILES — 1962
A crisis in the East/West protracted conflict

Background:
There were three crisis actors in the most ominous of all East/West crises, the United States, Cuba, and the Soviet Union. The crisis lasted from 16 October to 20 November 1962. Pre-crisis After the 1961 Bay of Pigs abortive invasion (see Case #181), Cuba became one of the central issues of U.S. foreign policy: the U.S. viewed Cuba as a potential source of communist-oriented subversive activities in Latin America. When the U.S. discovered the presence of Soviet military personnel in Cuba on 7 September 1962 it called up 150,000 reservists. The Soviets mobilized on the 11th. Although persistent rumors circulated concerning the deployment of Soviet missiles in Cuba, Soviet Ambassador Anatoly Dobrynin denied the charges, and Premier Khrushchev gave his personal assurances that ground-to-ground missiles would never be shipped to Cuba. On the eve of the Missile crisis Washington did not openly challenge the Soviet statements concerning the defensive character of the weapons being sent to Cuba.

Summary:
The U.S. crisis was triggered on 16 October when the CIA presented to President Kennedy photographic evidence of the presence of Soviet missiles in Cuba. The U.S. responded with a decision on the 20th to blockade all offensive military equipment en route to Cuba. When this was announced on 22 October, a crisis was triggered for Cuba and the USSR. An urgent meeting of the UN Security Council was requested by both the U.S. and Cuba on the 22nd, and by the USSR the next day. On the 23rd as well, the Soviets accused the United States of violating the UN Charter and announced an alert of its armed forces and those of the Warsaw Pact members. That day Cuba responded by condemning the U.S. blockade and declaring its willingness to fight.

A resolution was adopted on the 23rd by the OAS calling for the withdrawal of the missiles from Cuba and recommending that member-states take all measures, including the use of force, to ensure that the government of Cuba would not continue to receive military material. On 24 October the Security Council adopted a resolution requesting the Secretary-General to confer with the parties. On that same day, U Thant began mediation by sending identical letters to Khrushchev and Kennedy which proposed that the Soviet Union and the United States enter into negotiations, during which period both the shipment of arms and the quarantine would be suspended.

Moscow's major response to the crisis was a letter from Khrushchev to Kennedy on 26 October offering the removal of Soviet offensive weapons from Cuba and the cessation of further shipments in exchange for an end to the U.S. quarantine and a U.S. assurance that it would not invade Cuba. The situation was exacerbated on the 27th when a U.S. U-2 surveillance plane was shot down. That day another Khrushchev letter was received in Washington offering the removal of Soviet missiles from Cuba in exchange for the removal of U.S. missiles from Turkey. U.S. mobilization and aerial reconnaissance flights were stepped up. And on the 27th President Kennedy sent the Soviet premier an acceptance of the proposals contained in the letter of 26 October while making no reference to Khrushchev's second letter of the 27th. The following day Khrushchev notified the U.S. government that he had ordered work on the missile sites in Cuba stopped. He agreed to ship the missiles back to the USSR and promised that UN observers would be allowed to verify the dismantling of the sites. At the same time he warned Washington that U-2 reconnaissance flights over Cuba must be stopped as well.

The crisis continued at a lower level of intensity for several more weeks due to Cuban President Castro's demands concerning a U.S. pledge not to invade his country. On 30 October U Thant began talks in Havana, and Kennedy agreed to lift the quarantine for the duration of the talks. When Cuba rejected UN inspection, the U.S. resumed the quarantine and air surveillance.

The Kremlin sent Deputy Premier Anastas Mikoyan to Cuba on 2 November to try to persuade Castro to allow UN inspection. When this proved unsuccessful, a U.S.-USSR agreement was reached on 7 November allowing U.S. inspection and interception of Soviet ships leaving Cuba and the photographing of the missiles. The following day the superpowers negotiated the removal of the IL-28 bombers which Castro had claimed were Cuban property. Castro's agreement was conveyed to the U.S. on 20 November 1962, which terminated the Missile crisis for all three actors. The U.S. naval quarantine was lifted immediately, but aerial surveillance continued until the agreement was completely carried out.

U Thant's mediation efforts had an important impact on crisis resolution. The acting Secretary-General managed to act as a facilitator and kept lines of communication open by contacting both parties. He also used formulative mediation by highlighting common interests and making substantive suggestions to both sides for diffusing the crisis. Overall, U Thant's mediation effort was a significant factor in easing tensions between crisis actors which contributed to a more rapid termination of the crisis.

Three regional/security organizations were involved in this crisis--the OAS, NATO, and the WTO. Kennedy sent Dean Acheson to Paris on 23 October to brief NATO's Permanent Council on U.S. measures against Cuba. Involved were members of the Warsaw Pact, whose forces were put on alert, and Latin American states which offered military assistance to the U.S.

References:
Abel 1966; Allison 1971; Allyn, Blight, and Welch 1989-90; Bender 1975; Beschloss 1991; Blight and Welch 1989; Brenner and Blight 1995; Brinkley 1992; Brzezinski and Huntington 1964; Bundy 1988; Chayes 1974; Crankshaw 1966; Daniel and Hubbel 1963; Dinerstein 1976; Divine 1971; Dobrynin 1995; Draper 1965; Garthoff 1989, 1995; George 1991; George and Smoke 1974; Goldenberg 1965; Gromyko 1989; Halper 1971; Hershberg 1995; Hilsman 1967; Horelick 1964; Kennedy 1962, 1964; Kennedy 1969; Khrushchev 1970; Langley 1970; Larson 1963; Lebow and Stein 1994; Pachter 1963; Plank 1967; Rush 1970; Rusk 1990; Sagan 1985; Schlesinger 1965, 1978; Schoenbaum 1988; Shapley 1993; Sorensen 1965; Wainhouse 1966; Wohlstetter and Wohlstetter 1965; Young 1968; Zubok 1995.

Assistant:"""

In [18]:
output = generator.generate_simple(prompt, max_new_tokens = 200)

print(output[len(prompt):])

 The countries directly mentioned in the text are Cuba, Soviet Union (USSR), United States of America (USA). The organizations mentioned are Organization of American States (OAS), North Atlantic Treaty Organization (NATO), Warsaw Pact, Western Treaty Organization (WTO). The UN Security Council is also indirectly mentioned as a mediator between the
